In [ ]:
# Upload the repo manually
import os.path as osp; assert osp.exists('./ift-6269-vq-vae')
!if [ ! -d ift-6269-vq-vae ]; then unzip ift-6269-vq-vae.zip ;fi
!pip install ./ift-6269-vq-vae

In [ ]:
import argparse
import os
import os.path as osp
import torchvision
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from vqvae import VQVAE, VQVAEBase, VQVAEPrior, create_indices_dataset
from main import save_results

lr = 1e-3
prior_lr = 1e-3
batch_size = 128
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
epochs = 20
prior_epochs = 200



# TODO: Warning, no validation split here
# No normalization here since it is done in the code
transform = transforms.Compose([transforms.ToTensor()])
# Stupid code here
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform).data
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform).data

trainloader = DataLoader(train_data.transpose(0, 3, 1, 2).astype(np.float32), batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data.transpose(0, 3, 1, 2).astype(np.float32), batch_size=batch_size, shuffle=False)
vqvae_base = VQVAEBase(beta=1.)
vqvae_base = vqvae_base.to(device)
vqvae_trainer = Trainer(vqvae_base, trainloader, testloader, lr, device, epochs, print_every=1, grad_clip=1.)
vqvae_train_log, vqvae_test_log = vqvae_trainer.train()

prior_train_data = create_indices_dataset(trainloader, vqvae_base, device)
prior_test_data = create_indices_dataset(testloader, vqvae_base, device)
prior_trainloader = DataLoader(prior_train_data, batch_size=batch_size, shuffle=True)
prior_testloader = DataLoader(prior_test_data, batch_size=batch_size, shuffle=False)
vqvae_prior = VQVAEPrior(image_shape=(8, 8, 1), channel_ordered=False, n_colors=128, n_layers=8, n_filters=64)
vqvae_prior = vqvae_prior.to(device)
prior_trainer = Trainer(vqvae_prior, prior_trainloader, prior_testloader, prior_lr, device, prior_epochs, print_every=1)
prior_train_log, prior_test_log = prior_trainer.train()


vqvae_train_loss = vqvae_train_log['loss']
vqvae_test_loss = vqvae_test_log['loss']

prior_train_loss = prior_train_log['loss']
prior_test_loss = prior_test_log['loss']


vqvae = VQVAE(base=vqvae_base, prior=vqvae_prior)
samples = vqvae.sample(100)
# samples = np.random.rand(100, 32, 32, 3)


# # (100, C, H, W)
# samples = model.sample(100)

# assert torch.all((0 <= samples) & (samples <= 255))
samples = samples.cpu().numpy().transpose(0, 2, 3, 1).astype(np.uint8)

# Reconstruction
images = torch.from_numpy(test_data[:50].transpose(0, 3, 1, 2).astype(np.float32)).to(device)
recon = vqvae.reconstruct(images)
recon = recon.cpu().numpy().transpose(0, 2, 3, 1)

# (100, C, H, W), uint8
real_recon = np.concatenate((test_data[:50], recon), axis=0).astype(np.uint8)
save_results(samples, real_recon, vqvae_train_loss, vqvae_test_loss, prior_train_loss, prior_test_loss, show_figure=False)
